In [32]:
import requests
import json
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
from urllib.request import urlopen

from bs4 import BeautifulSoup
from html_table_parser import parser_functions as parser
from xml.etree.ElementTree import parse

import re
import openpyxl

## 카드 주소리스트 구하기
- 주소가 숨김처리 되어 있어 첫 html만 수동크롤링

In [33]:
tmp = '''<ul class="list"><li class="item"><a class="anchor" href="#"></a><div class="preview"><a class="anchor" href="#"><figure class="figure"><img class="img" src="/resource/images/card/SK/10061/SK_10061_20210616-101429_ver.png" alt=""></figure></a><a class="apply" href="https://tivan.naver.com/cac/uCPLJs1hNCywjO4RT-tN4qldahNRYgYYDg1YUxsrlny2KwB2HJLeQUKhqI8Sta8DenvMSauX7tp7PGaT_sH9V9O6lXEeRdKvIn9C1L49yugCcmBEDBP9w6_C_hgCDV8w0lb7kicwYzuYmCvO3RX-ejsxI5PqzpUVQ9d79E_jj9-CzqM-3cJu-2rl62_WCwrZE7gF_m1J-j8Ekl370RQgxeAd8jX6i0QCLQxDzCxDTkVkn2xtNFUuTQM1v1oyrhiXadoKUEsogmqKcatv6fts2t3cYQkT0QpmXnoOmTBmwbo=?ce=register" target="_blank">카드신청</a></div><div class="info"><a class="anchor" href="#"><b class="name">모두의 신세계 하나카드</b></a><p class="desc">신세계 백화점, 신세계 아카데미,</p><i class="annual_fee">국내 1만2천원, 해외 1만2천원</i><span class="rewards"><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_12.png" alt="쇼핑">쇼핑</i><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_103.png" alt="간편결제">간편결제</i></span></div><img src="https://tivan.naver.com/cai/a83T7xp0BS9lWobQjVDyepx27MA_-8MvtQKXOzHe8LESeAbGdl9TLAjWef3xh0N33CIQz-BLGNHziwEAjfqc3TiSBr4K-VryLzx8n4q5UfcfWb9flpKqslXkoVPtkJ4IR52CkNVLjCLRunYRa_02aGEZ4VW92lCl2um2VPCcUeRNV1IsgixR2IWznmhU5KoEoM4-3Q-J5y_bt1WtUpHhcr-fziLmmBcDRI1AkyxDV3i5i_jR0qa_WJzpNM2ZMKgI9GFDI-spRp2B2NPuTUXqTH_fs2DVGtDytLEn3hyZJQKjCyHdV7LW0mONEVXNpdjUvBfJTNcBFejCbSd35aK8-U8j3WFRqRpbTSTb3iKxMPO92syTXBouCmn_pjL0raTK" width="1" height="1" alt="" style="position:absolute"></li><li class="item"><a class="anchor" href="#"></a><div class="preview"><a class="anchor" href="#"><figure class="figure"><img class="img" src="/resource/images/card/SK/10019/SK_10019_20210119-134247_ver.png" alt=""></figure></a><a class="apply" href="https://tivan.naver.com/cac/uCPLJs1hNCywjO4RT-tN4qldahNRYgYYDg1YUxsrlny2KwB2HJLeQUKhqI8Sta8DenvMSauX7tp7PGaT_sH9V3eX6cCDzf98saoRiIhYbkpdtwM_2Q-efPSI2_7I_ukl3xSR--DqoUnMGRa9M9mRfHzMOFrKDix77ZuUseQqqdt4h9kWTCGXrPBGTsBqS3pLdr1SlEaNy2DAvWesFLb-XR7Nq0xUC3tRQldYEuysz2br4DTv_cmJj3KBAlEG9ACke05ogwZ9ZFKqLPU5cNceHv6npGpUQf2yESf3nYXZEOA=?ce=register" target="_blank">카드신청</a></div><div class="info"><a class="anchor" href="#"><b class="name">MULTI ON</b></a><p class="desc">전월 실적, 한도 없이 하나머니 적립</p><i class="annual_fee">국내 1만2천원, 해외 1만2천원</i><span class="rewards"><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_83.png" alt="언제나할인">언제나할인</i><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_103.png" alt="간편결제">간편결제</i></span></div></li><li class="item"><a class="anchor" href="#"></a><div class="preview"><a class="anchor" href="#"><figure class="figure"><img class="img" src="/resource/images/card/SK/10021/SK_10021_20210119-145655_ver.png" alt=""></figure></a><a class="apply" href="https://tivan.naver.com/cac/uCPLJs1hNCywjO4RT-tN4qldahNRYgYYDg1YUxsrlny2KwB2HJLeQUKhqI8Sta8DenvMSauX7tp7PGaT_sH9V3vi76A8-JgJp7xlWh-G-lNr44Tx4LyVuuAKg8q7NXGc3xSR--DqoUnMGRa9M9mRfHzMOFrKDix77ZuUseQqqdt4h9kWTCGXrPBGTsBqS3pLdr1SlEaNy2DAvWesFLb-XRodwYPfcufc-NUUxp6sUuXr4DTv_cmJj3KBAlEG9ACkLnlB-0NFBNF3-aprc9hwR_VcD_B8IYZUwZ0eto5jfFs=?ce=register" target="_blank">카드신청</a></div><div class="info"><a class="anchor" href="#"><b class="name">MULTI Living</b></a><p class="desc">가족소비 특화 할인 카드!</p><i class="annual_fee">국내 1만2천원, 해외 1만2천원</i><span class="rewards"><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_12.png" alt="쇼핑">쇼핑</i><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_7.png" alt="대형마트">대형마트</i></span></div></li><li class="item"><a class="anchor" href="#"></a><div class="preview"><a class="anchor" href="#"><figure class="figure"><img class="img" src="/resource/images/card/SK/10023/SK_10023_20210119-145835_ver.png" alt=""></figure></a><a class="apply" href="https://tivan.naver.com/cac/uCPLJs1hNCywjO4RT-tN4qldahNRYgYYDg1YUxsrlny2KwB2HJLeQUKhqI8Sta8DenvMSauX7tp7PGaT_sH9V8c1rEyMimLfkcoCMdX-VffloCJd3JY8LFBuSFwZiJqg3xSR--DqoUnMGRa9M9mRfHzMOFrKDix77ZuUseQqqdt4h9kWTCGXrPBGTsBqS3pLdr1SlEaNy2DAvWesFLb-XdZBvwYvb5M3STG680FhSLfr4DTv_cmJj3KBAlEG9ACkyiODCyeoa63qKbpSL8cQlmZOc-9c_PC-Wf-JyorUNuI=?ce=register" target="_blank">카드신청</a></div><div class="info"><a class="anchor" href="#"><b class="name">MULTI Young</b></a><p class="desc">간편결제 1% 청구 할인!</p><i class="annual_fee">국내 1만2천원, 해외 1만2천원</i><span class="rewards"><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_103.png" alt="간편결제">간편결제</i><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_16.png" alt="편의점">편의점</i></span></div></li><li class="item"><a class="anchor" href="#"></a><div class="preview"><a class="anchor" href="#"><figure class="figure"><img class="img" src="/resource/images/card/SK/4056/SK_4056_hor.PNG" alt=""></figure></a><a class="apply" href="https://tivan.naver.com/cac/uCPLJs1hNCywjO4RT-tN4qldahNRYgYYDg1YUxsrlny2KwB2HJLeQUKhqI8Sta8DenvMSauX7tp7PGaT_sH9V3S9PNaT9ZAymdV3GSkNaJA62jZhGP8KqF0neSZoIVnc5W24OyPWNFDoBtzacRHpPOGTuMVJaslOXuv73qrQQr6yb06IiHkF578zUCqEbMde2CUknxk2NWVhbifaV1-OsMWAwm55aCytLAUHWbr5PP-CirWsK2hLz4i_D2EO5KwyxNiMEuN5J34cRpnUr8-7lRjD0kAJFQ5EUhEVQBoR81U=?ce=register" target="_blank">카드신청</a></div><div class="info"><a class="anchor" href="#"><b class="name">하나 국민행복카드</b></a><p class="desc">임신, 출산 진료비 60만원까지 지원</p><i class="annual_fee">국내전용 없음, 해외겸용 없음</i><span class="rewards"><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_1001.png" alt="국민행복카드">국민행복카드</i><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_21.png" alt="의료">의료</i></span></div></li><li class="item"><a class="anchor" href="#"></a><div class="preview"><a class="anchor" href="#"><figure class="figure"><img class="img" src="/resource/images/card/SK/98/SK_98_hor.png" alt=""></figure></a><a class="apply" href="https://tivan.naver.com/cac/uCPLJs1hNCywjO4RT-tN4qldahNRYgYYDg1YUxsrlny2KwB2HJLeQUKhqI8Sta8DenvMSauX7tp7PGaT_sH9VxJ8yaD7L5oThfV6TbEwAW9dOU-Q4FperEiwKhn6hhGmKrKlR3YVoBn7ODBJRHzFOJoPdtlBPM2Enjcl-NIiW65qCMDNTN1WFwdA03OcO7Sjc82DWvyzAW6dWN5h9BBK7wH1UjjA4KF1KCaitRToLLS58s47KzySa_9R2LZzdD1pW-OjWhxreAiUqd_Vhjk_vN7XpRZssMxPfCszt0oJNp0=?ce=register" target="_blank">카드신청</a></div><div class="info"><a class="anchor" href="#"><b class="name">CLUB SK (클럽SK)</b></a><p class="desc">SKT 통신비 최대 1만5천원 할인</p><i class="annual_fee">국내 6천원, 해외 7천원</i><span class="events"><i class="tag">신규 최대 7천원 지급</i></span><span class="rewards"><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_163.png" alt="연회비지원">연회비지원</i><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_10.png" alt="통신">통신</i></span></div></li><li class="item"><a class="anchor" href="#"></a><div class="preview"><a class="anchor" href="#"><figure class="figure"><img class="img" src="/resource/images/card/SK/10017/SK_10017_20210204-173938_ver.png" alt=""></figure></a><a class="apply" href="https://tivan.naver.com/cac/uCPLJs1hNCywjO4RT-tN4qldahNRYgYYDg1YUxsrlny2KwB2HJLeQUKhqI8Sta8DenvMSauX7tp7PGaT_sH9V9r2_TxaiERuyzZ8dwi66HkoErejEm5_3a2YMhnoY9T63xSR--DqoUnMGRa9M9mRfHzMOFrKDix77ZuUseQqqdt4h9kWTCGXrPBGTsBqS3pLdr1SlEaNy2DAvWesFLb-XcEKvZrAgUPC-A8sveOqp8BS9BHh5HLCThsHojE43JKqP3aswMogSxFJLXndIbGsUr8TE5kCf1zNyOpcMQVDyCg=?ce=register" target="_blank">카드신청</a></div><div class="info"><a class="anchor" href="#"><b class="name">MULTI Any</b></a><p class="desc">전월 실적, 한도 없이 하나머니 적립</p><i class="annual_fee">국내 1만2천원, 해외 1만2천원</i><span class="rewards"><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_83.png" alt="언제나할인">언제나할인</i><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_103.png" alt="간편결제">간편결제</i></span></div></li><li class="item"><a class="anchor" href="#"></a><div class="preview"><a class="anchor" href="#"><figure class="figure"><img class="img" src="/resource/images/card/SK/3976/SK_3976_hor.png" alt=""></figure></a><a class="apply" href="https://tivan.naver.com/cac/uCPLJs1hNCywjO4RT-tN4qldahNRYgYYDg1YUxsrlny2KwB2HJLeQUKhqI8Sta8DenvMSauX7tp7PGaT_sH9V0zQUMfXcL1cFdpoFn8XYag62jZhGP8KqF0neSZoIVnc5W24OyPWNFDoBtzacRHpPOGTuMVJaslOXuv73qrQQr6yb06IiHkF578zUCqEbMdepB2W6EtxoifDgVD6w80fQ8WAwm55aCytLAUHWbr5PP-60E5czvX_VX7-iNgTKhcsxNiMEuN5J34cRpnUr8-7lT1Q9oK6l96AsJto46QCKu4=?ce=register" target="_blank">카드신청</a></div><div class="info"><a class="anchor" href="#"><b class="name">모두의 건강</b></a><p class="desc">대형마트 5% 하나머니 적립</p><i class="annual_fee">국내 1만5천원, 해외 1만5천원</i><span class="events"><i class="tag">신규 최대 1만5천원 지급</i></span><span class="rewards"><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_163.png" alt="연회비지원">연회비지원</i><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_14.png" alt="외식">외식</i></span></div></li><li class="item"><a class="anchor" href="#"></a><div class="preview"><a class="anchor" href="#"><figure class="figure"><img class="img" src="/resource/images/card/SK/10025/SK_10025_20210119-160858_ver.png" alt=""></figure></a><a class="apply" href="https://tivan.naver.com/cac/uCPLJs1hNCywjO4RT-tN4qldahNRYgYYDg1YUxsrlny2KwB2HJLeQUKhqI8Sta8DenvMSauX7tp7PGaT_sH9V3eauYGhS9RuF2t02EOV3fsJRJjQnzinEHxP8sVV-Eoe3xSR--DqoUnMGRa9M9mRfHzMOFrKDix77ZuUseQqqdt4h9kWTCGXrPBGTsBqS3pLdr1SlEaNy2DAvWesFLb-XZ2MihF42CWXTxoHEp0zWg7r4DTv_cmJj3KBAlEG9ACkd6r2PVXLAbY7xb5gz9ilIz2BC-J4Xtqn9oQENccOTsQ=?ce=register" target="_blank">카드신청</a></div><div class="info"><a class="anchor" href="#"><b class="name">MULTI Oil</b></a><p class="desc">자동차 특화 혜택 카드!</p><i class="annual_fee">국내 1만5천원, 해외 1만5천원</i><span class="rewards"><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_1.png" alt="주유">주유</i><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_13.png" alt="카페/베이커리">카페/베이커리</i></span></div></li><li class="item"><a class="anchor" href="#"></a><div class="preview"><a class="anchor" href="#"><figure class="figure"><img class="img" src="/resource/images/card/SK/2349/SK_2349_hor.png" alt=""></figure></a><a class="apply" href="https://tivan.naver.com/cac/uCPLJs1hNCywjO4RT-tN4qldahNRYgYYDg1YUxsrlny2KwB2HJLeQUKhqI8Sta8DenvMSauX7tp7PGaT_sH9V8yRujRRvgz4Qb4VhDNYqVte6SjIdjMymA1emEp6NFIEGdDbaOG9bTPRNgLpKmtzDGuENzkW9V7MDvp04eqhMHxNZlhm7o7R6NnS8fQgFwfxrOiUfcs3k0RB86bG72y8q2m-PfUzC1c3JzoUYB5pcemrz_skP28QXqQR7N1sFvG7odgBeQ9JhIkLuvsTHIapfwA2CjAOmFAfa0BFrdO-HKM=?ce=register" target="_blank">카드신청</a></div><div class="info"><a class="anchor" href="#"><b class="name">Mile 1.6 대한항공</b></a><p class="desc">국내 1,500원당 1.6 마일 적립</p><i class="annual_fee">국내전용 없음, 해외 4만5천원</i><span class="rewards"><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_4.png" alt="항공마일리지">항공마일리지</i><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_13.png" alt="카페/베이커리">카페/베이커리</i></span></div></li><li class="item"><a class="anchor" href="#"></a><div class="preview"><a class="anchor" href="#"><figure class="figure"><img class="img" src="/resource/images/card/SK/2631/SK_2631_hor.png" alt=""></figure></a><a class="apply" href="https://tivan.naver.com/cac/kjpZSNcOU4H7V7AeFvL3MQ436KVg3GjptRvcX2bt7nf5rxEE3doDFzvAHQaBgGEXowO67ZuC02L5YqG5Pz_spW3xvUUz8J9ao0usXG7uN1MVknVo9IEF3SOq1ZNeaP_4TGYfbeb2P-xj45240OM1KZJTLai41g0-FwotQ9DLRZEHTv4kzyIJS3qt7yxMtjDHo-vRyhQni9kXIdxTwYlDceoS-NdvH2djxMYqcYT_ZmwSzsKt8uQofLawfG1cz0oo8fftAx7nd-cbUcINjWeJx-SwACbaDslTdBFsfP2EfIfo1ahzjmLIwK-Ah3egVdyjZkb15HcZZk2BEWR8kmZJ1g==?ce=register" target="_blank">카드신청</a></div><div class="info"><a class="anchor" href="#"><b class="name">Any PLUS (애니플러스)</b></a><p class="desc">국내 가맹점 0.7% 할인</p><i class="annual_fee">국내전용 없음, 해외 1만5천원</i><span class="events"><i class="tag">신규 최대 1만7천원 지급</i></span><span class="rewards"><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_163.png" alt="연회비지원">연회비지원</i><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_12.png" alt="쇼핑">쇼핑</i></span></div><img src="https://tivan.naver.com/cai/gruH4t0fR33mUCGaKOhsAnqqEw9cK6uYBmzgtbG2vU5qWoNFg3CabzNQaA5ajAMMWfk_8ihzC8VjWGaqIvysnc-9nfrHNX-Zeo9o0BXbU7Bp0iDAJFdKsZk78BRe_73b-DB5Pf9pXyLQLlOOCISQMtDS-52lL0H924K06T7pDOParSN9z2yYXHnkL-aVy7l1Bzt03OyyQYZcOxboxb3QqJg_AtJWwIucPk6DHjTReGywssc3Q-ShVeTOVx0_WsdOTs_5AUhQkwMJXQXjo651S7Qa7mwObD9EYeVXzyfFM5vdWhx-YS6GBaUIZKUfuopzZdtMcnF5P0Fl4azQSLUDHkqLd5Gg-Vqq_syW0cQGYxl8Lt4P3k0QEAaSDRH-f9ou7gZeM3gmbHlwo6Fi9wVuPASxp2sLw_-x2qaq4tn__2DnYi1mnatH8mzq1uVl9GFCUP4a6yM0iykif-i5akxALDLNw50jDxy2ruOAaIuss3hvsP9cLRUnkc4C1NBLNJz12B3r7yoJkmVPsjqu0JiJhayKLUvSarJXGhhrVPTR8Cm_XyHO0DXLdQgi1N5hnjSTZ7jmq6YC2pUWUdCQ2oFMIQ==" width="1" height="1" alt="" style="position: absolute;"></li><li class="item"><a class="anchor" href="#"></a><div class="preview"><a class="anchor" href="#"><figure class="figure"><img class="img" src="/resource/images/card/SK/2778/SK_2778_hor.png" alt=""></figure></a><a class="apply" href="https://tivan.naver.com/cac/kjpZSNcOU4H7V7AeFvL3MQ436KVg3GjptRvcX2bt7nf5rxEE3doDFzvAHQaBgGEXowO67ZuC02L5YqG5Pz_spW3xvUUz8J9ao0usXG7uN1MVknVo9IEF3SOq1ZNeaP_4mGcGnreZmQli7A6H3NwZgrLOC52_U-S0y27VL8-vkfAD0_VKFsOxTp-14HmLwumVRHT85fhl5jQ1JhXOiPchUa3zuQ1aoYHrAaytgxVzAzmZsUSUx5ytyHyWes70L67Qa86hm3jNe4ZJUt187GQI2BTYSIeja1x6a885-wPjqCbIQkiHPAZnFhe_f9LCk6HTt3Hz-ktaF6F2XbK4TDtVpA==?ce=register" target="_blank">카드신청</a></div><div class="info"><a class="anchor" href="#"><b class="name">My Trip SKYPASS 카드</b></a><p class="desc">대한항공 마일리지 적립한도 없이 적립</p><i class="annual_fee">국내전용 없음, 해외 3만원</i><span class="rewards"><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_4.png" alt="항공마일리지">항공마일리지</i><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_19.png" alt="레저">레저</i></span></div></li><li class="item"><a class="anchor" href="#"></a><div class="preview"><a class="anchor" href="#"><figure class="figure"><img class="img" src="/resource/images/card/SK/2334/SK_2334_hor.png" alt=""></figure></a><a class="apply" href="https://tivan.naver.com/cac/kjpZSNcOU4H7V7AeFvL3MQ436KVg3GjptRvcX2bt7nf5rxEE3doDFzvAHQaBgGEXowO67ZuC02L5YqG5Pz_spW3xvUUz8J9ao0usXG7uN1MVknVo9IEF3SOq1ZNeaP_4iktr97siDhULMyd1zOYmOFJ6ipCctPsrn4LPNJHjXxcHTv4kzyIJS3qt7yxMtjDHo-vRyhQni9kXIdxTwYlDceoS-NdvH2djxMYqcYT_Zmxj3fLFWZqJFhlGoNQMTbST8fftAx7nd-cbUcINjWeJx4erlaWyJ6Rv65fhUo2UkovwAGUPtMH3X2ijGouA8DuOA35aWAbaALMpvxoUPcRXqA==?ce=register" target="_blank">카드신청</a></div><div class="info"><a class="anchor" href="#"><b class="name">1Q Shopping+ (원큐 쇼핑플러스)</b></a><p class="desc">쇼핑 적립 월 최대 4만 하나머니</p><i class="annual_fee">국내전용 없음, 해외 1만원</i><span class="events"><i class="tag">신규 최대 1만2천원 지급</i></span><span class="rewards"><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_163.png" alt="연회비지원">연회비지원</i><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_13.png" alt="카페/베이커리">카페/베이커리</i></span></div></li><li class="item"><a class="anchor" href="#"></a><div class="preview"><a class="anchor" href="#"><figure class="figure"><img class="img" src="/resource/images/card/SK/2779/SK_2779_hor.png" alt=""></figure></a><a class="apply" href="https://tivan.naver.com/cac/kjpZSNcOU4H7V7AeFvL3MQ436KVg3GjptRvcX2bt7nf5rxEE3doDFzvAHQaBgGEXowO67ZuC02L5YqG5Pz_spW3xvUUz8J9ao0usXG7uN1MVknVo9IEF3SOq1ZNeaP_4uE4ZA6c4VP5AQmygMgVSxFtogmA-JMsIhs52jIHdukmTDntT0bIcHx9HEFWyCHdvbTUaeLzWMtxdxeVbK6eCHEVuCOuYOQIxsiofE1D9VZ-Zi9aEWjYcGIVlFY1cuSxmasDUPxjuKID5W0fdKK-wu3o2Cso8VdAglNa4tRJDKmnmDQMzA31MfoXso8uXdYOKV0GFEmfPWiiZ9PMRaWamxA==?ce=register" target="_blank">카드신청</a></div><div class="info"><a class="anchor" href="#"><b class="name">My Trip AsianaClub 카드</b></a><p class="desc">아시아나항공마일리지 한도 없이 적립</p><i class="annual_fee">국내전용 없음, 해외 3만원</i><span class="rewards"><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_4.png" alt="항공마일리지">항공마일리지</i><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_19.png" alt="레저">레저</i></span></div></li><li class="item"><a class="anchor" href="#"></a><div class="preview"><a class="anchor" href="#"><figure class="figure"><img class="img" src="/resource/images/card/SK/2355/SK_2355_hor.png" alt=""></figure></a><a class="apply" href="https://tivan.naver.com/cac/kjpZSNcOU4H7V7AeFvL3MQ436KVg3GjptRvcX2bt7nf5rxEE3doDFzvAHQaBgGEXowO67ZuC02L5YqG5Pz_spW3xvUUz8J9ao0usXG7uN1MVknVo9IEF3SOq1ZNeaP_4UKXfx383PuwdGmAd-34bryUEIHsu9-l-PQfxja3ykomTDntT0bIcHx9HEFWyCHdvbTUaeLzWMtxdxeVbK6eCHEVuCOuYOQIxsiofE1D9VZ9mbn8j1BMosbYY1RmKzbAtTLK2CERc-a5ZWk3Ugm0ldkwl51VDVHgalawZhOMkuAqjArnDQJUntzcILa_kmzGO9OKPT4w1KKLpNlkoY_05gg==?ce=register" target="_blank">카드신청</a></div><div class="info"><a class="anchor" href="#"><b class="name">통커 카드</b></a><p class="desc">통신 10% 할인, 커피 50% 할인</p><i class="annual_fee">국내전용 없음, 해외 1만7천원</i><span class="events"><i class="tag">신규 최대 2만원 지급</i></span><span class="rewards"><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_163.png" alt="연회비지원">연회비지원</i><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_12.png" alt="쇼핑">쇼핑</i></span></div></li><li class="item"><a class="anchor" href="#"></a><div class="preview"><a class="anchor" href="#"><figure class="figure"><img class="img" src="/resource/images/card/SK/2716/SK_2716_hor.png" alt=""></figure></a><a class="apply" href="https://tivan.naver.com/cac/kjpZSNcOU4H7V7AeFvL3MQ436KVg3GjptRvcX2bt7nf5rxEE3doDFzvAHQaBgGEXowO67ZuC02L5YqG5Pz_spW3xvUUz8J9ao0usXG7uN1MVknVo9IEF3SOq1ZNeaP_4ir-9b8qWKQrIvX76pmLy7rSiKSZzFqUK72fRN2htiduTDntT0bIcHx9HEFWyCHdvbTUaeLzWMtxdxeVbK6eCHEVuCOuYOQIxsiofE1D9VZ8JX03MFpIMmevHBXENxMVUasDUPxjuKID5W0fdKK-wu3o2Cso8VdAglNa4tRJDKml2LwA11F8ZzKPTJY9ggZv8AGm6orYSe4NWJjRqz0TXkQ==?ce=register" target="_blank">카드신청</a></div><div class="info"><a class="anchor" href="#"><b class="name">CLUB Signature SKYPASS</b></a><p class="desc">라운지키, 외식,면세점 이용권 제공</p><i class="annual_fee">국내전용 없음, 해외 14만5천원</i><span class="rewards"><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_6.png" alt="프리미엄">프리미엄</i><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_5.png" alt="Priority Pass">Priority Pass</i></span></div></li><li class="item"><a class="anchor" href="#"></a><div class="preview"><a class="anchor" href="#"><figure class="figure"><img class="img" src="/resource/images/card/SK/2390/SK_2390_hor.png" alt=""></figure></a><a class="apply" href="https://tivan.naver.com/cac/kjpZSNcOU4H7V7AeFvL3MQ436KVg3GjptRvcX2bt7nf5rxEE3doDFzvAHQaBgGEXowO67ZuC02L5YqG5Pz_spW3xvUUz8J9ao0usXG7uN1MVknVo9IEF3SOq1ZNeaP_41Ov7bHp1Mirx0N5uRQOjPSroWtzwPDYyAu4lNuTAbXOTDntT0bIcHx9HEFWyCHdvbTUaeLzWMtxdxeVbK6eCHEVuCOuYOQIxsiofE1D9VZ9VvE6HxiLPx9uDY1CSYN-eTLK2CERc-a5ZWk3Ugm0ldr668mHAMdJyA56w_vnYkqKdtN_W_oZNuQAmf2MidU2qV9v5RVsG0TiSSLw4La79BA==?ce=register" target="_blank">카드신청</a></div><div class="info"><a class="anchor" href="#"><b class="name">#tag1카드 Navy (태그원카드 네이비)</b></a><p class="desc">학원,아파트관리비 10% 우대 할인</p><i class="annual_fee">국내전용 없음, 해외 1만2천원</i><span class="events"><i class="tag">신규 최대 1만2천원 지급</i></span><span class="rewards"><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_163.png" alt="연회비지원">연회비지원</i><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_10.png" alt="통신">통신</i></span></div></li><li class="item"><a class="anchor" href="#"></a><div class="preview"><a class="anchor" href="#"><figure class="figure"><img class="img" src="/resource/images/card/SK/1928/SK_1928_hor.png" alt=""></figure></a><a class="apply" href="https://tivan.naver.com/cac/kjpZSNcOU4H7V7AeFvL3MQ436KVg3GjptRvcX2bt7nf5rxEE3doDFzvAHQaBgGEXowO67ZuC02L5YqG5Pz_spW3xvUUz8J9ao0usXG7uN1MVknVo9IEF3SOq1ZNeaP_4zCEYANxKIt8YyHCCvLw21G9UDSlQ6bN_NBNrlGmJgBoHTv4kzyIJS3qt7yxMtjDHo-vRyhQni9kXIdxTwYlDceoS-NdvH2djxMYqcYT_ZmyYrT_nCwrQfobXNcJJW64k8fftAx7nd-cbUcINjWeJx-ndO3HAcofXAgdevwn_MK3onqrISjGSzvL90XrDxHsqDMfx3CG7Vu7ls1EEAjpr-g==?ce=register" target="_blank">카드신청</a></div><div class="info"><a class="anchor" href="#"><b class="name">하나멤버스 1Q카드 내맘대로</b></a><p class="desc">다른 서비스가 필요할땐 내맘대로 변경</p><i class="annual_fee">국내전용 없음, 해외 1만원</i><span class="rewards"><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_14.png" alt="외식">외식</i><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_12.png" alt="쇼핑">쇼핑</i></span></div></li><li class="item"><a class="anchor" href="#"></a><div class="preview"><a class="anchor" href="#"><figure class="figure"><img class="img" src="/resource/images/card/SK/3517/SK_3517_hor.png" alt=""></figure></a><a class="apply" href="https://tivan.naver.com/cac/kjpZSNcOU4H7V7AeFvL3MQ436KVg3GjptRvcX2bt7nf5rxEE3doDFzvAHQaBgGEXowO67ZuC02L5YqG5Pz_spW3xvUUz8J9ao0usXG7uN1MVknVo9IEF3SOq1ZNeaP_4sD-0bT_yX6PqzlmxP6CSzbKTzJQb-3a5-RtZBiUMrrKztz-1U2be6x8d1jBSl8j_NUC97BxYNkJi3XVJfjrJOF3S0A97HBMTPZtytHUkID2po6_0VBsWUxXB9xCxJvuebwgb8Eo0LtBHqjIaFauXeP5IHr_slP6_5QYirUBEieSLbT8eI0bAJE8nIgANLG8YTGryEkgwPb17A4QfrQMh-A==?ce=register" target="_blank">카드신청</a></div><div class="info"><a class="anchor" href="#"><b class="name">모두의 쇼핑</b></a><p class="desc">온라인 쇼핑10%,생활쇼핑 5% 적립</p><i class="annual_fee">국내 1만5천원, 해외 1만5천원</i><span class="events"><i class="tag">신규 최대 1만5천원 지급</i></span><span class="rewards"><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_163.png" alt="연회비지원">연회비지원</i><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_12.png" alt="쇼핑">쇼핑</i></span></div></li><li class="item"><a class="anchor" href="#"></a><div class="preview"><a class="anchor" href="#"><figure class="figure"><img class="img" src="/resource/images/card/SK/2346/SK_2346_hor.png" alt=""></figure></a><a class="apply" href="https://tivan.naver.com/cac/kjpZSNcOU4H7V7AeFvL3MQ436KVg3GjptRvcX2bt7nf5rxEE3doDFzvAHQaBgGEXowO67ZuC02L5YqG5Pz_spW3xvUUz8J9ao0usXG7uN1MVknVo9IEF3SOq1ZNeaP_4IkK1nH2Of1A3aHepD7TR94w5PHS_pXpDUY1XO-qoWtTG7ptWo5Bz0VGDU7aW1HADlE6bdhEfEaunJR_0X6YcCBgZesYz7Q1VjQbe4vH2Z_T07knbDBig8EB7BL6TvSLd666WI1k4YhCewoeuPC6aIw8wfnS8g51_yZRjtyLe1HSh_cyAiRNl5NNg6zKPw4W2I0POTiBSt_uq6vF5953clg==?ce=register" target="_blank">카드신청</a></div><div class="info"><a class="anchor" href="#"><b class="name">1Q Special+ (원큐 스페셜플러스)</b></a><p class="desc">스페셜 적립 월 최대 4만 머니</p><i class="annual_fee">국내전용 없음, 해외 1만원</i><span class="rewards"><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_12.png" alt="쇼핑">쇼핑</i><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_7.png" alt="대형마트">대형마트</i></span></div></li><li class="item"><a class="anchor" href="#"></a><div class="preview"><a class="anchor" href="#"><figure class="figure"><img class="img" src="/resource/images/card/SK/2489/SK_2489_hor.jpg" alt=""></figure></a><a class="apply" href="https://tivan.naver.com/cac/QoBupM2L9vLqHQY36gWAc5nXYwmHFEXL4t6CG-BBNR0H_it_HQmp0Lh0OgaD1bL2Q9CAPqlgM0CVyLrXr0uQrpVtMK0df-lgS4Xy4UwbHEsnQsDAGJyc3mLNmL5uBPHnkfysNNQLO0-MEfrMcddxGrAcXxhfiqohaEmIvXIZKWYD25Kh9mskOsNG3DDzjwFE1E_y70fd8F5MKVTVID_sB72uTpn12f1gXKVy4bu0BodTPpe-CFB6TP8An7-g8nO3YEaJFu_A5DjtTkGzcn_swz4_Doig7c2IpHolJ8JW--tXrEfursJ8OcI5CiCZEoLnsIl7kNarlxz3QLjqICR9Ag==?ce=register" target="_blank">카드신청</a></div><div class="info"><a class="anchor" href="#"><b class="name">CLUB PREMIER 카드 [Travel]</b></a><p class="desc">국내 6대 저가항공 50% 청구할인</p><i class="annual_fee">국내전용 없음, 해외 20만원</i><span class="rewards"><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_6.png" alt="프리미엄">프리미엄</i><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_12.png" alt="쇼핑">쇼핑</i></span></div><img src="https://tivan.naver.com/cai/xOWq9gGvXpVaQ0bgCv44-tb-pzllTsu9TpLyQ6jOwgN8LbM1J_g4W8HgbxLnIRV4ekdiLXKrthDS9hTUKLLzZAXzag2xC_MEXXcymK6sWPSKu_GremaSE-2Xp6T93gDF1fXi2YO2tuigi3rUwiKluZV7rx05LrVpAjFZvnsEmWuZTKb3mul5PhVWCLNWVTwuKEXJZp_nFJp5paYJO6q3bz_3UTNxT_xg04ZxfYGvHo01WzWGKvrL_rU4TE2vnM3KW5dmmnPd2nDCmRP6YIq1GUMpEEBk_CJ8sYDyfiwJNkAynpoFeudClsZWbSG6iVAa9VTOy7eayZOX7ZEbDg84HzJXtEdWxihovgCSEbpRcKcqsSjQnIqVy7HV_t824f3r8PiWE3ZkB0vL9jktQwORvOSY9WjVwsK89m35vIMk0Dusdyruo9fgBhx5v3aGN5IWfFJG_oq1S-wcF-A-zdAxof9g6OykeBynx3yC8Ml5XhtGn6h2aunuPcNV7wojhPu9cQuYfI6d9p16KqCaJru6SqsWSdbO32TJtYTJXWqmh_-s3uu9tuQ6iNJB-4-wg0O0" width="1" height="1" alt="" style="position: absolute;"></li><li class="item"><a class="anchor" href="#"></a><div class="preview"><a class="anchor" href="#"><figure class="figure"><img class="img" src="/resource/images/card/SK/2737/SK_2737_hor.png" alt=""></figure></a><a class="apply" href="https://tivan.naver.com/cac/lQwB_BOB3BmOn3vhvVI6k9hYRSpUnqwFGvM84xlecvkwvrt5I3akj7CpaMjykK71mjdPf1-lCyoSxilhkeSze23xvUUz8J9ao0usXG7uN1MVknVo9IEF3SOq1ZNeaP_47bTpYel9LLbRAkvIP-Y6Tj_3DzHwc2Wl2PWMOd27A9mTDntT0bIcHx9HEFWyCHdvbTUaeLzWMtxdxeVbK6eCHEVuCOuYOQIxsiofE1D9VZ8qzibSzcbMgnR659dlybtDasDUPxjuKID5W0fdKK-wu3o2Cso8VdAglNa4tRJDKmkLnFjvRR1v6xH1db41JRxydt6T0coeueAdpsn1cFwFIw==?ce=register" target="_blank">카드신청</a></div><div class="info"><a class="anchor" href="#"><b class="name">CLUB Primus Skypass</b></a><p class="desc">항공권/주유권/면세점 교환권/PP택1</p><i class="annual_fee">국내전용 없음, 해외 10만원</i><span class="rewards"><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_4.png" alt="항공마일리지">항공마일리지</i><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_63.png" alt="바우처">바우처</i></span></div></li><li class="item"><a class="anchor" href="#"></a><div class="preview"><a class="anchor" href="#"><figure class="figure"><img class="img" src="/resource/images/card/SK/2333/SK_2333_hor.png" alt=""></figure></a><a class="apply" href="https://tivan.naver.com/cac/lQwB_BOB3BmOn3vhvVI6k9hYRSpUnqwFGvM84xlecvkwvrt5I3akj7CpaMjykK71mjdPf1-lCyoSxilhkeSze23xvUUz8J9ao0usXG7uN1MVknVo9IEF3SOq1ZNeaP_4wB-4hhL0fYIXW40qM2fe6wwDoMsBIrjCGjfhetbHIPQqsqVHdhWgGfs4MElEfMU4mg922UE8zYSeNyX40iJbrmoIwM1M3VYXB0DTc5w7tKMxksVoThBEpPyxRvzueDQAAfVSOMDgoXUoJqK1FOgstLjPp2L1lI9EDz2CV2kQGhZB9QScKAG6HFhEK5vcTcPoHhmbNJsZwV1PY2pAgORk-A==?ce=register" target="_blank">카드신청</a></div><div class="info"><a class="anchor" href="#"><b class="name">1Q Daily+ (원큐 데일리플러스)</b></a><p class="desc">매일매일 최대 1.0% 적립</p><i class="annual_fee">국내전용 없음, 해외 1만원</i><span class="events"><i class="tag">신규 최대 1만2천원 지급</i></span><span class="rewards"><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_15.png" alt="대중교통">대중교통</i><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_8.png" alt="관리비">관리비</i></span></div></li><li class="item"><a class="anchor" href="#"></a><div class="preview"><a class="anchor" href="#"><figure class="figure"><img class="img" src="/resource/images/card/SK/2490/SK_2490_hor.jpg" alt=""></figure></a><a class="apply" href="https://tivan.naver.com/cac/WoD1u5PBFrNyNZpGsLGlRr1GsrGPJJqVtLmulnpgWBgBoTXJJXWutVvBiw5FozfiCNlTM-RjAKVkakbIlsWkdo3MJH2bGxWMHlG1Vctz1pCMClGV_AXCly-T01wPF3mcfENVXqzdreVwFJDUN5YBF19dlzMqBvo0W8eynrKrz6Obx_LJZHLF4CV3AEcPv18FLRK-ouHrB2D5ZengMAHTPPptQn9qc8wto1XRS8s31W5hCmke6vqpWATV1Ls9TkRwhdb_lCUgNWUnDcyNKL2lykioy5cdajaZJJZqVsA-IO0zzAT6Uf2GbuO8szUkVyIUFrGmaaG2nbSm_prOjTgmew==?ce=register" target="_blank">카드신청</a></div><div class="info"><a class="anchor" href="#"><b class="name">CLUB PREMIER 카드 [Hotel]</b></a><p class="desc">국내 특급호텔 50% 청구할인</p><i class="annual_fee">국내전용 없음, 해외 20만원</i><span class="rewards"><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_6.png" alt="프리미엄">프리미엄</i><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_1.png" alt="주유">주유</i></span></div></li><li class="item"><a class="anchor" href="#"></a><div class="preview"><a class="anchor" href="#"><figure class="figure"><img class="img" src="/resource/images/card/SK/790/SK_790_hor.png" alt=""></figure></a><a class="apply" href="https://tivan.naver.com/cac/lQwB_BOB3BmOn3vhvVI6k9hYRSpUnqwFGvM84xlecvkwvrt5I3akj7CpaMjykK71mjdPf1-lCyoSxilhkeSze23xvUUz8J9ao0usXG7uN1MVknVo9IEF3SOq1ZNeaP_4_VxLP3Lvl3PwO3Ldje83wiG5oTd2DRlyRfBUVykKiLyTDntT0bIcHx9HEFWyCHdvbTUaeLzWMtxdxeVbK6eCHEVuCOuYOQIxsiofE1D9VZ9ciDZVvIUjAEA1fsZwLxxOTLK2CERc-a5ZWk3Ugm0ldm0KhA3ZHmwfWR0jDzgdmcXnTXo-t8Kg_Dk_XBiFcFbnnR_jjfrpIsVZ4Iz69iCW2g==?ce=register" target="_blank">카드신청</a></div><div class="info"><a class="anchor" href="#"><b class="name">Smart Any (스마트애니)</b></a><p class="desc">온라인 결제 시 최대 1.4% 할인</p><i class="annual_fee">국내 5천원, 해외 5천원</i><span class="events"><i class="tag">신규 최대 5천원 지급</i></span><span class="rewards"><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_163.png" alt="연회비지원">연회비지원</i><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_103.png" alt="간편결제">간편결제</i></span></div></li><li class="item"><a class="anchor" href="#"></a><div class="preview"><a class="anchor" href="#"><figure class="figure"><img class="img" src="/resource/images/card/SK/2389/SK_2389_hor.png" alt=""></figure></a><a class="apply" href="https://tivan.naver.com/cac/lQwB_BOB3BmOn3vhvVI6k9hYRSpUnqwFGvM84xlecvkwvrt5I3akj7CpaMjykK71mjdPf1-lCyoSxilhkeSze23xvUUz8J9ao0usXG7uN1MVknVo9IEF3SOq1ZNeaP_46FRACgap7c7iqRH9tOjuk3hnNDX5d7QfG2CeRCaT6mGTDntT0bIcHx9HEFWyCHdvbTUaeLzWMtxdxeVbK6eCHEVuCOuYOQIxsiofE1D9VZ__xfZnWL6Te7TnBy7Tj0eoTLK2CERc-a5ZWk3Ugm0ldr668mHAMdJyA56w_vnYkqIUt6oikxwnvA4cJCli47e8HsrtTVH8DTpaq-HIe-XX_w==?ce=register" target="_blank">카드신청</a></div><div class="info"><a class="anchor" href="#"><b class="name">#tag1카드 Orange (태그원카드 오렌지)</b></a><p class="desc">커피 50% 우대 할인</p><i class="annual_fee">국내전용 없음, 해외 1만2천원</i><span class="events"><i class="tag">신규 최대 1만2천원 지급</i></span><span class="rewards"><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_163.png" alt="연회비지원">연회비지원</i><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_10.png" alt="통신">통신</i></span></div></li><li class="item"><a class="anchor" href="#"></a><div class="preview"><a class="anchor" href="#"><figure class="figure"><img class="img" src="/resource/images/card/SK/1313/SK_1313_hor.png" alt=""></figure></a><a class="apply" href="https://tivan.naver.com/cac/lQwB_BOB3BmOn3vhvVI6k9hYRSpUnqwFGvM84xlecvkwvrt5I3akj7CpaMjykK71mjdPf1-lCyoSxilhkeSze23xvUUz8J9ao0usXG7uN1MVknVo9IEF3SOq1ZNeaP_4TYsxrHHOb02GVX844B59O9rVuoeIXd9019hbb4Nz8WCztz-1U2be6x8d1jBSl8j_Ctx1ywW8fF1dtLBb4V1-sF3S0A97HBMTPZtytHUkID0qbZkRP_nRI5VozQhczYQZKkZhEfL87wLl2f4Y4rTWWybWZBEVuKu6jI9AN3tzTnOeO3FIAE_8GuzGg_tEu-m7TcR1-me0gz6jkqYvs6o4Uw==?ce=register" target="_blank">카드신청</a></div><div class="info"><a class="anchor" href="#"><b class="name">하나 BC 그린카드 [전국형]</b></a><p class="desc">국내 전 가맹점 0.2~0.8% 적립</p><i class="annual_fee">국내전용 없음, 해외겸용 없음</i><span class="rewards"><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_83.png" alt="언제나할인">언제나할인</i><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_62.png" alt="포인트/캐시백">포인트/캐시백</i></span></div></li><li class="item"><a class="anchor" href="#"></a><div class="preview"><a class="anchor" href="#"><figure class="figure"><img class="img" src="/resource/images/card/SK/2420/SK_2420_hor.png" alt=""></figure></a><a class="apply" href="https://tivan.naver.com/cac/lQwB_BOB3BmOn3vhvVI6k9hYRSpUnqwFGvM84xlecvkwvrt5I3akj7CpaMjykK71mjdPf1-lCyoSxilhkeSze23xvUUz8J9ao0usXG7uN1MVknVo9IEF3SOq1ZNeaP_4lOClraTxSY26nYbaoFPQ7ExE-p1L03IO6GqSgUWDpVKTDntT0bIcHx9HEFWyCHdvbTUaeLzWMtxdxeVbK6eCHEVuCOuYOQIxsiofE1D9VZ9mi__kThU2iIX3pP1BECvFTLK2CERc-a5ZWk3Ugm0ldm9OQrmmKcEjAL_fpw7ySkP_znamUti1uYdsfPu7R8OOWumbsMI7iNU2AI7L-tFmug==?ce=register" target="_blank">카드신청</a></div><div class="info"><a class="anchor" href="#"><b class="name">my pass 마패 카드</b></a><p class="desc">대중교통 20% 할인(월1만5천원)</p><i class="annual_fee">국내전용 없음, 해외 1만7천원</i><span class="events"><i class="tag">신규 최대 1만7천원 지급</i></span><span class="rewards"><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_163.png" alt="연회비지원">연회비지원</i><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_16.png" alt="편의점">편의점</i></span></div></li><li class="item"><a class="anchor" href="#"></a><div class="preview"><a class="anchor" href="#"><figure class="figure"><img class="img" src="/resource/images/card/SK/2717/SK_2717_hor.png" alt=""></figure></a><a class="apply" href="https://tivan.naver.com/cac/lQwB_BOB3BmOn3vhvVI6k9hYRSpUnqwFGvM84xlecvkwvrt5I3akj7CpaMjykK71mjdPf1-lCyoSxilhkeSze23xvUUz8J9ao0usXG7uN1MVknVo9IEF3SOq1ZNeaP_4gJSo6aHU1ZLf4SXr1aPUYFjTV81oE6QDFD0OKwxKrlaTDntT0bIcHx9HEFWyCHdvbTUaeLzWMtxdxeVbK6eCHEVuCOuYOQIxsiofE1D9VZ-OfKFdkmT9CKZnKECMnz7GasDUPxjuKID5W0fdKK-wu3o2Cso8VdAglNa4tRJDKmlrQbbJAvLilqUNQm1ooDgd1ZWqe4AnENI72c2DbQvt_g==?ce=register" target="_blank">카드신청</a></div><div class="info"><a class="anchor" href="#"><b class="name">CLUB Signature Asiana Club</b></a><p class="desc">라운지키, 외식 면세점 이용권 제공</p><i class="annual_fee">국내전용 없음, 해외 14만5천원</i><span class="rewards"><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_6.png" alt="프리미엄">프리미엄</i><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_5.png" alt="Priority Pass">Priority Pass</i></span></div></li><li class="item"><a class="anchor" href="#"></a><div class="preview"><a class="anchor" href="#"><figure class="figure"><img class="img" src="/resource/images/card/SK/2618/SK_2618_hor.png" alt=""></figure></a><a class="apply" href="https://tivan.naver.com/cac/lQwB_BOB3BmOn3vhvVI6k9hYRSpUnqwFGvM84xlecvkwvrt5I3akj7CpaMjykK71mjdPf1-lCyoSxilhkeSze23xvUUz8J9ao0usXG7uN1MVknVo9IEF3SOq1ZNeaP_45-8fg1sbh13hRkKU_8LclipVgoGP9_QC0lCqOPTgoq4D0_VKFsOxTp-14HmLwumVRHT85fhl5jQ1JhXOiPchUa3zuQ1aoYHrAaytgxVzAznjLyhW74A5tF-bMBfWaChTEK97pIz_7aBm2P3P_B01UGrKDiUmm3zNqmnJbUZWmvShH4RFjqs_TwtzzG-EZRJDkWIc99mnpAbE-RRAAGOz3g==?ce=register" target="_blank">카드신청</a></div><div class="info"><a class="anchor" href="#"><b class="name">1Q Global VIVA (원큐 글로벌 비바)</b></a><p class="desc">해외 결제 브랜드 수수료(1%) 면제</p><i class="annual_fee">국내 1만3천원, 해외 1만5천원</i><span class="events"><i class="tag">신규 최대 1만5천원 지급</i></span><span class="rewards"><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_163.png" alt="연회비지원">연회비지원</i><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_15.png" alt="대중교통">대중교통</i></span></div></li><li class="item"><a class="anchor" href="#"></a><div class="preview"><a class="anchor" href="#"><figure class="figure"><img class="img" src="/resource/images/card/SK/2350/SK_2350_hor.png" alt=""></figure></a><a class="apply" href="https://tivan.naver.com/cac/iJSGbtbooLeD0nUnUBRLtwfHQpYvM3Pvt2gOoHy22UqdF08ewUWbXFsWUJ3FBRgrB5PAr-ckRTs7q6PPRrmjOoJ5dztmgP0awBHdXfYAwaSgp9EoDIWGLs-BQyz4vdK_-WjJG3dQk4cnqtfqtm_Dx7x9ACZae5-4Ou0SjIRTP-UHTv4kzyIJS3qt7yxMtjDHo-vRyhQni9kXIdxTwYlDceoS-NdvH2djxMYqcYT_Zmwp6lbwWcHTzlYVR7nWQaxI8fftAx7nd-cbUcINjWeJxzXT0ZmTKci55JyhxQoKZ-uNe-_4fE_KctzGGvFykE3m0UT3vQxNKpgjQ-EPGh9Nyg==?ce=register" target="_blank">카드신청</a></div><div class="info"><a class="anchor" href="#"><b class="name">Mile 1.6 아시아나</b></a><p class="desc">국내 1,500원당 1.6 마일 적립</p><i class="annual_fee">국내전용 없음, 해외 4만5천원</i><span class="rewards"><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_4.png" alt="항공마일리지">항공마일리지</i><i class="reward"><img class="icon" src="/resource/images/icons/benefits/bene_13.png" alt="카페/베이커리">카페/베이커리</i></span></div><img src="https://tivan.naver.com/cai/POAVgVZ310MP5Qeaecr9FgJu4DQxe3hqEnLoClBDexbnprkfqNL20vRBm2shFvC5JJHI_fSm3Yro-x5f0shnjVPRcq1Zy7gF8e1mdZSUMgVNOoESCrKYDrCZNYo_2mNh8JmFPkKJ_fuAdBEu9ANkpR6FXBd0FRvdIXTw1KmuVCaeKkvy6tDeoItjOKS_gAs5xPN2WcKgA4dDGxFHOGMnNgWunGHZGZk5C-0VKQWgG1Hgjli8hZt0-7LNjXijUbCvY3TEbKUhnbGyyBV1L649gFLN4Qsa8sQrLBz3YXDU6CcF2_YJFuwmit8A3Hf7vGSaHp5Ydghebqyo_wcfCNnIUV-N21e6UfMdop66Cc5ddVsdPba4emheH_L8GBMNGv6mZh5DdMyrv6bib8vgPxQsdD-fsS5HP8ZXEyKu87Uz0SjLqjyuJJIb2-thN6rzAlUJ3X1HfQMbAtuNM2xFTux6w4sr-N7Eg4OImgsitOfLL4pfGj-SZT5wrMskTm0S9_bn" width="1" height="1" alt="" style="position: absolute;"></li></ul>'''

In [34]:
url = 'https://card-search.naver.com/list?companyCode=SK&sortMethod=ri&isRefetch=true'

In [35]:
html = requests.get(url)

In [36]:
soup = BeautifulSoup(tmp, 'html.parser')

In [37]:
soup_list = soup.select('.img')

In [38]:
hana_creditcard = []
for i in soup_list:
    hana_creditcard.append(str(i).split('/')[5])

## 카테고리, 카드이름 구하기

In [39]:
category=set()
hana_creditcard_name = []

for i in hana_creditcard:
    url = 'https://card-search.naver.com/item?cardAdId='+i
    
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    
    #카드이름
    cardNameParse = soup.select("b.txt")
    cardName = cardNameParse[0].text
    hana_creditcard_name.append(str(cardName))
    
    for j in range(len(soup.select('.details'))):
        category_temp = soup.select('.details')[j].select("b.text")[0].text
        category.add(category_temp)
        
        
category = list(category)

In [40]:
card_df = pd.DataFrame(index=hana_creditcard_name, columns=category)

In [41]:
category=set()
hana_creditcard_name = []
for i in hana_creditcard:
    url = 'https://card-search.naver.com/item?cardAdId='+i
    
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    
    #카드이름
    cardNameParse = soup.select("b.txt")
    cardName = cardNameParse[0].text
    hana_creditcard_name.append(str(cardName))
    
    for j in range(len(soup.select('.details'))):
        category_temp = soup.select('.details')[j].select("b.text")[0].text
        category.add(category_temp)
        
        
        temp = str(soup.select('.details')[j].select("i")[0]).replace('<br/>','\n')
        result_soup = BeautifulSoup(temp, 'html.parser')
        
        result = str(result_soup.text)[0:len(result_soup.text)-1]
        
        card_df.loc[cardName, category_temp] = result
        
        
        
category = list(category)

In [42]:
i=hana_creditcard[0]

In [43]:
url = 'https://card-search.naver.com/item?cardAdId='+i

In [44]:
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

#카드이름
cardNameParse = soup.select("b.txt")
cardName = cardNameParse[0].text
hana_creditcard_name.append(str(cardName))

In [45]:
category_temp = soup.select('.details')[j].select("b.text")[0].text

In [46]:
category_temp

'카페/베이커리'

In [47]:
temp_df = pd.DataFrame(columns=['a','b','c'])

In [48]:
temp_df.append({'a':1,'b':1,'c':4}, ignore_index=True)

,a,b,c
0,1,1,4


In [49]:
card_benefits=pd.DataFrame(columns=['CARD_CODE', 'CATE'])

In [50]:
soup.select('.details')

[<details class="details"><summary class="summary"><h5 class="name"><b class="text">쇼핑</b><i class="desc">백화점, 아카데미 5% 청구 할인<br/>시코르닷컴 5% 청구 할인<br/></i></h5></summary><div class="detail"><dl class="list"><dt class="detail_title">백화점, 아카데미 5% 청구 할인</dt><dt class="detail_title">시코르닷컴 5% 청구 할인</dt></dl></div></details>,
 <details class="details"><summary class="summary"><h5 class="name"><b class="text">간편결제</b><i class="desc">페이결제 5% 청구할인 제공<br/></i></h5></summary><div class="detail"><dl class="list"><dt class="detail_title">페이결제 5% 청구할인 제공</dt></dl></div></details>,
 <details class="details"><summary class="summary"><h5 class="name"><b class="text">카페/베이커리</b><i class="desc">카페 20% 청구 할인<br/>베이커리 20% 청구 할인<br/></i></h5></summary><div class="detail"><dl class="list"><dt class="detail_title">카페 20% 청구 할인</dt><dt class="detail_title">베이커리 20% 청구 할인</dt></dl></div></details>,
 <details class="details"><summary class="summary"><h5 class="name"><b class="text">대중교통</b><i class="desc">택시 20% 청구

In [51]:
temp = list(soup.select('.summary')[0].select('.desc')[0])

In [52]:
temp

['백화점, 아카데미 5% 청구 할인', <br/>, '시코르닷컴 5% 청구 할인', <br/>]

In [53]:
soup.select('.details')[j].select("i")[0]

<i class="desc">카페 20% 청구 할인<br/>베이커리 20% 청구 할인<br/></i>

In [54]:
card_df.reset_index(inplace=True)

In [55]:
card_df.columns

Index(['index', '관리비', '문화', '포인트/캐시백', '뷰티', '외식', '통신', '프리미엄', '편의점',
       '카페/베이커리', '렌탈', '대형마트', '오토', '레저', '바우처', '국민행복카드', '항공마일리지',
       'Priority Pass', '대중교통', '쇼핑', '연회비지원', '간편결제', '주유', '의료', '금융',
       '언제나할인', '교육', '영화'],
      dtype='object')

In [56]:
card_df=card_df.rename(columns={'index' : 'CARD_NAME', '교육' : 'EDUCATION', '오토' : 'AUTOMOBILES', '쇼핑' : 'SHOPPING', '레저' : 'LEISURE', 'Priority Pass' : 'PRIORITY_PASS',\
                       '금융':'FINANCE', '항공마일리지' : 'AIR_MILEAGE', '뷰티' : 'BEAUTY', '통신' : 'COMMUNICATION', '외식' : 'EAT_OUT', '국민행복카드' : 'NATIONAL_VOUCHER',\
                       '문화' : 'CULTURE', '카페/베이커리' : 'CAFE_BAKERY', '의료' : 'MEDICAL', '프리미엄' : 'PREMIUM', '연회비지원' : 'ANNUAL_FEE_SUPPORT', '바우처' : 'VOUCHER',\
                       '언제나할인' : 'ALWAYS_DISCOUNT', '대중교통' : 'PUBLIC_TRANSPORTATION', '영화' : 'MOVIE', '포인트/캐시백' : 'POINT_CASHBACK', '간편결제':'EASY_PAY', '대형마트' : 'HYPERMART',\
                       '렌탈':'RENTAL', '주유':'OIL','관리비':'MAINTENANCE', '편의점':'CVS'})

In [57]:
card_df.to_excel('result.xlsx', index=False)

In [58]:
card_df

,CARD_NAME,MAINTENANCE,CULTURE,POINT_CASHBACK,BEAUTY,EAT_OUT,COMMUNICATION,PREMIUM,CVS,CAFE_BAKERY,...,PUBLIC_TRANSPORTATION,SHOPPING,ANNUAL_FEE_SUPPORT,EASY_PAY,OIL,MEDICAL,FINANCE,ALWAYS_DISCOUNT,EDUCATION,MOVIE
0,모두의 신세계 하나카드,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,카페 20% 청구 할인\n베이커리 20% 청구 할인,...,택시 20% 청구 할인,"백화점, 아카데미 5% 청구 할인\n시코르닷컴 5% 청구 할인",NaN,페이결제 5% 청구할인 제공,NaN,NaN,NaN,NaN,NaN,NaN
1,MULTI ON,NaN,디지털 구독 4% 적립,NaN,NaN,딜리버리 3% 적립,NaN,NaN,NaN,NaN,...,NaN,온라인쇼핑 2% 적립\n쿠팡 로켓와우 구독 4% 적립,NaN,페이결제 1% 적립,NaN,NaN,NaN,전 가맹점 0.7% 적립,NaN,NaN
2,MULTI Living,(매일) 전기/도시가스요금 5% 할인,NaN,NaN,NaN,NaN,(매일) 통신요금 자동이체 5% 할인,NaN,NaN,NaN,...,NaN,(주중) 온라인쇼핑 10% 할인\n(주말) 백화점 10% 할인,NaN,(매일) 페이결제 1% 할인,(주말) 주유/LPG 10% 할인,NaN,NaN,NaN,NaN,NaN
3,MULTI Young,전기/도시가스요금 10% 청구 할인,디지털 구독 30% 청구 할인,NaN,NaN,딜리버리 10% 청구 할인,통신요금 자동이체 10% 청구 할인,NaN,편의점 10% 청구 할인,NaN,...,"버스, 지하철 10% 청구 할인 제공",NaN,NaN,페이결제 1% 청구 할인 제공,NaN,NaN,NaN,NaN,NaN,NaN
4,하나 국민행복카드,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,커피 전문점 10% 청구할인,...,NaN,NaN,NaN,NaN,NaN,"병원, 약국 업종 5% 청구할인\n산모, 신생아 건강관리 지원사업\n여성청소년 위생...",NaN,NaN,보육료/유아학비 할인 서비스,NaN
5,CLUB SK (클럽SK),NaN,NaN,OK캐쉬백포인트 적립 및 사용,NaN,"아웃백,TGIF 최대 10%할인\n영화/외식/커피/차량 서비스제공조건",SKT 통신비 최대 1만5천원 할인\nT membership 서비스 제공\n이동통신...,NaN,NaN,커피전문점 최대 5% 할인\n영화/외식/커피/차량 서비스제공조건,...,대중교통 최대 7% 할인\n대중교통 서비스 제공 조건,신라면세점 VIP멤버십 제공,신규 회원 연회비 캐쉬백 이벤트!,NaN,SK주유 리터당 최대 150원 할인\nSK충전소 리터당 최대 70원 할인\n주유/충...,NaN,하나은행 자동화기기 수수료 10회면제,NaN,학원 업종 최대 10% 할인\n마트/학원 서비스 제공 조건,영화 최대 3천원 할인\n영화/외식/커피/차량 서비스제공조건
6,MULTI Any,NaN,NaN,NaN,NaN,"배달의민족,요기요 3% 하나머니 적립",NaN,NaN,NaN,NaN,...,"버스, 지하철 4% 하나머니 적립",백화점 2% 하나머니 적립,NaN,페이결제 1% 하나머니 적립,NaN,NaN,NaN,전가맹점 0.7% 하나머니 적립,NaN,NaN
7,모두의 건강,NaN,NaN,NaN,NaN,저녁식사 5% 하나머니 적립,NaN,NaN,NaN,커피 5% 하나머니 적립,...,NaN,온라인 5% 하나머니 적립,신규 회원 연회비 캐쉬백 이벤트!,NaN,NaN,병원 5% 하나머니 적립,NaN,NaN,NaN,NaN
8,MULTI Oil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,카페 5% 청구할인,...,NaN,NaN,NaN,페이결제 1% 청구 할인,주유/LPG 10% 할인,NaN,NaN,NaN,NaN,NaN
9,Mile 1.6 대한항공,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,스타벅스 4천원 청구할인,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"CGV, 롯데시네마 4천원 청구할인"


In [16]:
import os



NameError: name 'os' is not defined

In [19]:
card = pd.read_csv('card.csv')

In [22]:
temp = card[card['CARD_CATEGORY']=='CREDIT']

In [24]:
temp

,CARD_CODE,CARD_NAME,CARD_CATEGORY,ANNUAL_FEE
0,1001,모두의 신세계 하나카드,CREDIT,12000
1,1002,MULTI ON,CREDIT,12000
2,1003,MULTI Living,CREDIT,12000
3,1004,MULTI Young,CREDIT,12000
4,1005,하나 국민행복카드,CREDIT,0
5,1006,CLUB SK (클럽SK),CREDIT,6000
6,1007,MULTI Any,CREDIT,12000
7,1008,모두의 건강,CREDIT,15000
8,1009,MULTI Oil,CREDIT,15000
9,1010,Mile 1.6 대한항공,CREDIT,45000


In [29]:
temp.iloc[0,0]

1001

In [31]:
for i in range(len(temp)):
    file_oldname = os.path.join("img", str(temp.iloc[i,0])+'.png')
    file_newname_newfile = os.path.join("img", temp.iloc[i,1]+'.png')

    os.rename(file_oldname, file_newname_newfile)